In [1]:
import pandas as pd
import numpy as np
import os

from bs4 import BeautifulSoup
from selenium import webdriver

In [6]:
df = download_stock_codes('kospi')

In [7]:
code_list = df['종목코드']

In [8]:
chrome_driver_path = os.path.abspath("./chromedriver.exe")
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'plugins': 2, 'popups': 2,
                                                    'geolocation': 2, 'notifications': 2,
                                                    'auto_select_certificate': 2, 'fullscreen': 2,
                                                    'mouselock': 2, 'mixed_script': 2, 'media_stream': 2,
                                                    'media_stream_mic': 2, 'media_stream_camera': 2,
                                                    'protocol_handlers': 2, 'ppapi_broker': 2,
                                                    'automatic_downloads': 2, 'midi_sysex': 2,
                                                    'push_messaging': 2, 'ssl_cert_decisions': 2,
                                                    'metro_switch_to_desktop': 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2,
                                                    'site_engagement': 2, 'durable_storage': 2,
                              
                                                    'javascript': 2}}

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
# options.add_argument(
#     "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90  Safari/537.36")

driver = webdriver.Chrome(executable_path=chrome_driver_path,
                          chrome_options=options)

c:\users\jay\pycharmprojects\data_science\venv\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: use options instead of chrome_options


In [9]:
# driver.page_source -> 개발자 코드와 동일

In [11]:
samsung_list = get_shareholder_list("010170")

400


In [12]:
samsung_list

['티에프오인더스트리', '설윤석', '양귀애', '설윤성']

In [5]:
import urllib.parse

MARKET_CODE_DICT = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
}

DOWNLOAD_URL = 'kind.krx.co.kr/corpgeneral/corpList.do'

def download_stock_codes(market=None, delisted=False):
    params = {'method': 'download'}

    if market.lower() in MARKET_CODE_DICT:
        params['marketType'] = MARKET_CODE_DICT[market]

    if not delisted:
        params['searchType'] = 13

    params_string = urllib.parse.urlencode(params)
    request_url = urllib.parse.urlunsplit(['http', DOWNLOAD_URL, '', params_string, ''])

    df = pd.read_html(request_url, header=0)[0]
    df.종목코드 = df.종목코드.map('{:06d}'.format)

    return df

In [10]:
def get_shareholder_list(code: str):
    """
    code: 종목코드
    return:
    shareholder_list: type=list, 대주주리스트
    """
    driver.get('https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}'.format(code))

    count = 10
    pg_list = []
    while True:
        try:
            driver.find_element_by_xpath("//a[@class='btn_moreE sJJ' and @data-grp='"+ str(count) + "']").click()
            pg_list.append(count)
            count += 10
        except:
            print(400)
            break

    html = driver.page_source

    data = BeautifulSoup(html, 'html.parser')

    shareholder_list = []

    for i in pg_list:
        refined_data = data.find_all('tr', {'class':"c_sJJ{}".format(i) +" c_sJJ"})
        for i in range(len(refined_data)):
            shareholder_list.append(refined_data[i].find('span').text)
    return shareholder_list